In [1]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType,FloatType

warehouse_location = 'hdfs://hdfs-nn:9870/TrabalhoPL'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
hdfs_path = "hdfs://hdfs-nn:9000/TrabalhoPL/bronze/Air_Quality.csv"

customSchema = StructType([
    StructField("Unique ID", StringType(), True),        
    StructField("Indicator ID", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Measure", StringType(), True),
    StructField("Measure Info", StringType(), True),
    StructField("Geo Type Name", StringType(), True),
    StructField("Geo Join ID", StringType(), True),
    StructField("Geo Place Name", StringType(), True), 
    StructField("Time Period", StringType(), True),  
    StructField("Start_Date", StringType(), True),
    StructField("Data Value", FloatType(), True), 
    StructField("Message", StringType(), True)
])

projeto_air = spark \
            .read\
            .option("delimiter",",")\
            .option("header","true")\
            .schema(customSchema) \
            .csv(hdfs_path)
projeto_air.toPandas()

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,Message
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2013,06/01/2013,34.639999,None
1,216499,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2014,06/01/2014,33.220001,None
2,219969,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2013,06/01/2013,31.250000,None
3,219970,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2014,06/01/2014,31.150000,None
4,164876,383,Sulfur Dioxide (SO2),Mean,ppb,CD,211,Morris Park and Bronxdale (CD11),Winter 2008-09,12/01/2008,5.890000,None
...,...,...,...,...,...,...,...,...,...,...,...,...
16117,671118,386,Ozone (O3),Mean,ppb,CD,306,Park Slope and Carroll Gardens (CD6),Summer 2020,06/01/2020,28.700001,None
16118,671119,386,Ozone (O3),Mean,ppb,CD,305,East New York and Starrett City (CD5),Summer 2020,06/01/2020,29.559999,None
16119,671120,386,Ozone (O3),Mean,ppb,CD,304,Bushwick (CD4),Summer 2020,06/01/2020,29.650000,None
16120,671121,386,Ozone (O3),Mean,ppb,CD,303,Bedford Stuyvesant (CD3),Summer 2020,06/01/2020,29.280001,None


In [3]:
replaced_projeto_air = projeto_air.drop("Message")
replaced_projeto_air.toPandas()

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2013,06/01/2013,34.639999
1,216499,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2014,06/01/2014,33.220001
2,219969,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2013,06/01/2013,31.250000
3,219970,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2014,06/01/2014,31.150000
4,164876,383,Sulfur Dioxide (SO2),Mean,ppb,CD,211,Morris Park and Bronxdale (CD11),Winter 2008-09,12/01/2008,5.890000
...,...,...,...,...,...,...,...,...,...,...,...
16117,671118,386,Ozone (O3),Mean,ppb,CD,306,Park Slope and Carroll Gardens (CD6),Summer 2020,06/01/2020,28.700001
16118,671119,386,Ozone (O3),Mean,ppb,CD,305,East New York and Starrett City (CD5),Summer 2020,06/01/2020,29.559999
16119,671120,386,Ozone (O3),Mean,ppb,CD,304,Bushwick (CD4),Summer 2020,06/01/2020,29.650000
16120,671121,386,Ozone (O3),Mean,ppb,CD,303,Bedford Stuyvesant (CD3),Summer 2020,06/01/2020,29.280001


In [4]:
replaced_projeto_air2 = replaced_projeto_air.withColumn('End_Date',
                                            when(col("Time Period").startswith('Summer'), concat(split('Start_Date', '/').getItem(2), lit('/09/'), split('Start_Date', '/').getItem(1)))
                                            .when(col("Time Period").startswith('Winter'), concat(substring(split('Start_Date', '/').getItem(2)+1, 0, 4), lit('/03/'), split('Start_Date', '/').getItem(1)))
                                            .when(col("Time Period").startswith('2') & col('Time Period').contains('-'), concat(lit('20'), substring('Time Period', 8,50), lit('/12/'), lit('31')))
                                            .when(col("Time Period").startswith('Annual'), concat(substring('Time Period', 16,50), lit('/12/'), lit('/31')))
                                            .otherwise(concat(col('Time Period'), lit('/12'), lit('/31'))))
replaced_projeto_air2.toPandas()

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,End_Date
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2013,06/01/2013,34.639999,2013/09/01
1,216499,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2014,06/01/2014,33.220001,2014/09/01
2,219969,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2013,06/01/2013,31.250000,2013/09/01
3,219970,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2014,06/01/2014,31.150000,2014/09/01
4,164876,383,Sulfur Dioxide (SO2),Mean,ppb,CD,211,Morris Park and Bronxdale (CD11),Winter 2008-09,12/01/2008,5.890000,2009/03/01
...,...,...,...,...,...,...,...,...,...,...,...,...
16117,671118,386,Ozone (O3),Mean,ppb,CD,306,Park Slope and Carroll Gardens (CD6),Summer 2020,06/01/2020,28.700001,2020/09/01
16118,671119,386,Ozone (O3),Mean,ppb,CD,305,East New York and Starrett City (CD5),Summer 2020,06/01/2020,29.559999,2020/09/01
16119,671120,386,Ozone (O3),Mean,ppb,CD,304,Bushwick (CD4),Summer 2020,06/01/2020,29.650000,2020/09/01
16120,671121,386,Ozone (O3),Mean,ppb,CD,303,Bedford Stuyvesant (CD3),Summer 2020,06/01/2020,29.280001,2020/09/01


In [5]:
replaced_projeto_air3 = replaced_projeto_air2.withColumn('Localidade',
                                             when ((col("Geo Type Name")=="CD") & (col("Geo Join ID")>=101) & (col("Geo Join ID")<=112),"Manhattan")
                                            .when ((col("Geo Type Name")=="CD") & (col("Geo Join ID")>=201) & (col("Geo Join ID")<=212),"Bronx") 
                                            .when ((col("Geo Type Name")=="CD") & (col("Geo Join ID")>=301) & (col("Geo Join ID")<=318),"Brooklyn")
                                            .when ((col("Geo Type Name")=="CD") & (col("Geo Join ID")>=401) & (col("Geo Join ID")<=414),"Queens")
                                            .when ((col("Geo Type Name")=="CD") & (col("Geo Join ID")>=501) & (col("Geo Join ID")<=503),"Staten Island")
                                            
                                            .when (((col("Geo Type Name")=="UHF42") | (col("Geo Type Name")=="UHF34")) & (col("Geo Join ID")>=301) & (col("Geo Join ID")<=310) ,"Manhattan") 
                                            .when (((col("Geo Type Name")=="UHF42") | (col("Geo Type Name")=="UHF34")) & (col("Geo Join ID")>=101) & (col("Geo Join ID")<=107),"Bronx") 
                                            .when (((col("Geo Type Name")=="UHF42") | (col("Geo Type Name")=="UHF34")) & (col("Geo Join ID")>=201) & (col("Geo Join ID")<=211),"Brooklyn")
                                            .when (((col("Geo Type Name")=="UHF42") | (col("Geo Type Name")=="UHF34")) & (col("Geo Join ID")>=401) & (col("Geo Join ID")<=410),"Queens")
                                            .when (((col("Geo Type Name")=="UHF42") | (col("Geo Type Name")=="UHF34")) & (col("Geo Join ID")>=501) & (col("Geo Join ID")<=504),"Staten Island") 
                                                         
                                            .when((col("Geo Type Name")=="Borough") & (col("Geo Join ID")==3),"Manhattan")
                                            .when ((col("Geo Type Name")=="Borough") & (col("Geo Join ID")==1),"Bronx") 
                                            .when ((col("Geo Type Name")=="Borough") & (col("Geo Join ID")==2),"Brooklyn")
                                            .when ((col("Geo Type Name")=="Borough") & (col("Geo Join ID")==4),"Queens")
                                            .when ((col("Geo Type Name")=="Borough") & (col("Geo Join ID")==5),"Staten Island")
                                            
                                            .otherwise("New York City"))

replaced_projeto_air3.toPandas()

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,End_Date,Localidade
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2013,06/01/2013,34.639999,2013/09/01,Brooklyn
1,216499,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2014,06/01/2014,33.220001,2014/09/01,Brooklyn
2,219969,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2013,06/01/2013,31.250000,2013/09/01,Bronx
3,219970,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2014,06/01/2014,31.150000,2014/09/01,Bronx
4,164876,383,Sulfur Dioxide (SO2),Mean,ppb,CD,211,Morris Park and Bronxdale (CD11),Winter 2008-09,12/01/2008,5.890000,2009/03/01,Bronx
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16117,671118,386,Ozone (O3),Mean,ppb,CD,306,Park Slope and Carroll Gardens (CD6),Summer 2020,06/01/2020,28.700001,2020/09/01,Brooklyn
16118,671119,386,Ozone (O3),Mean,ppb,CD,305,East New York and Starrett City (CD5),Summer 2020,06/01/2020,29.559999,2020/09/01,Brooklyn
16119,671120,386,Ozone (O3),Mean,ppb,CD,304,Bushwick (CD4),Summer 2020,06/01/2020,29.650000,2020/09/01,Brooklyn
16120,671121,386,Ozone (O3),Mean,ppb,CD,303,Bedford Stuyvesant (CD3),Summer 2020,06/01/2020,29.280001,2020/09/01,Brooklyn


In [6]:
replaced_projeto_air4 = replaced_projeto_air3.withColumn("Start_Date", to_date(col("Start_Date"), "MM/dd/yyyy")) \
                                            .withColumn("End_Date", to_date(col("End_Date"), "yyyy/MM/dd"))
replaced_projeto_air4 = replaced_projeto_air4.withColumn('Ano', (split(replaced_projeto_air4['Start_Date'], '-').getItem(0)).cast(IntegerType()))
replaced_projeto_air4.toPandas()

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,End_Date,Localidade,Ano
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2013,2013-06-01,34.639999,2013-09-01,Brooklyn,2013
1,216499,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2014,2014-06-01,33.220001,2014-09-01,Brooklyn,2014
2,219969,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2013,2013-06-01,31.250000,2013-09-01,Bronx,2013
3,219970,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2014,2014-06-01,31.150000,2014-09-01,Bronx,2014
4,164876,383,Sulfur Dioxide (SO2),Mean,ppb,CD,211,Morris Park and Bronxdale (CD11),Winter 2008-09,2008-12-01,5.890000,2009-03-01,Bronx,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16117,671118,386,Ozone (O3),Mean,ppb,CD,306,Park Slope and Carroll Gardens (CD6),Summer 2020,2020-06-01,28.700001,2020-09-01,Brooklyn,2020
16118,671119,386,Ozone (O3),Mean,ppb,CD,305,East New York and Starrett City (CD5),Summer 2020,2020-06-01,29.559999,2020-09-01,Brooklyn,2020
16119,671120,386,Ozone (O3),Mean,ppb,CD,304,Bushwick (CD4),Summer 2020,2020-06-01,29.650000,2020-09-01,Brooklyn,2020
16120,671121,386,Ozone (O3),Mean,ppb,CD,303,Bedford Stuyvesant (CD3),Summer 2020,2020-06-01,29.280001,2020-09-01,Brooklyn,2020


In [7]:
replaced_projeto_air5 = replaced_projeto_air4 \
    .withColumnRenamed("Unique ID","Unique_ID") \
    .withColumnRenamed("Indicator ID","Indicator_ID") \
    .withColumnRenamed("Measure Info","Measure_Info") \
    .withColumnRenamed("Geo Type Name","Geo_Type_Name") \
    .withColumnRenamed("Geo Join ID","Geo_Join_ID")\
    .withColumnRenamed("Geo Place Name","Geo_Place_Name")\
    .withColumnRenamed("Time Period","Time_Period") \
    .withColumnRenamed("Data Value","Data_Value")
replaced_projeto_air5.toPandas()

,Unique_ID,Indicator_ID,Name,Measure,Measure_Info,Geo_Type_Name,Geo_Join_ID,Geo_Place_Name,Time_Period,Start_Date,Data_Value,End_Date,Localidade,Ano
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2013,2013-06-01,34.639999,2013-09-01,Brooklyn,2013
1,216499,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2014,2014-06-01,33.220001,2014-09-01,Brooklyn,2014
2,219969,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2013,2013-06-01,31.250000,2013-09-01,Bronx,2013
3,219970,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2014,2014-06-01,31.150000,2014-09-01,Bronx,2014
4,164876,383,Sulfur Dioxide (SO2),Mean,ppb,CD,211,Morris Park and Bronxdale (CD11),Winter 2008-09,2008-12-01,5.890000,2009-03-01,Bronx,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16117,671118,386,Ozone (O3),Mean,ppb,CD,306,Park Slope and Carroll Gardens (CD6),Summer 2020,2020-06-01,28.700001,2020-09-01,Brooklyn,2020
16118,671119,386,Ozone (O3),Mean,ppb,CD,305,East New York and Starrett City (CD5),Summer 2020,2020-06-01,29.559999,2020-09-01,Brooklyn,2020
16119,671120,386,Ozone (O3),Mean,ppb,CD,304,Bushwick (CD4),Summer 2020,2020-06-01,29.650000,2020-09-01,Brooklyn,2020
16120,671121,386,Ozone (O3),Mean,ppb,CD,303,Bedford Stuyvesant (CD3),Summer 2020,2020-06-01,29.280001,2020-09-01,Brooklyn,2020


In [8]:
spark.sql(
    """
    DROP TABLE IF EXISTS Projeto.Tabela_Ar
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE Projeto.Tabela_Ar(
        Unique_ID VARCHAR(50),
        Indicator_ID int, 
        Name VARCHAR(500),
        Measure VARCHAR(50),
        Measure_Info VARCHAR(50),
        Geo_Type_Name VARCHAR(50),
        Geo_Join_ID VARCHAR(50),
        Geo_Place_Name VARCHAR(500),
        Time_Period VARCHAR(50),
        Start_Date DATE, 
        Data_Value float, 
        End_Date DATE, 
        Localidade VARCHAR(50)

    )
       USING DELTA
   
   PARTITIONED BY (
        Ano INT

    )
    LOCATION 'hdfs://hdfs-nn:9000/TrabalhoPL/silver/Projeto.db/Tabela_Ar'
    """
)

DataFrame[]

In [9]:
#write df to hive deltalake_table
replaced_projeto_air5 \
    .select("Unique_ID","Indicator_ID","Name","Measure","Measure_Info","Geo_Type_Name",
            "Geo_Join_ID","Geo_Place_Name","Time_Period", "Start_Date", "Data_Value", "End_Date", "Localidade" , "Ano") \
    .write \
    .mode("overwrite") \
    .partitionBy("Ano") \
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/TrabalhoPL/silver/Projeto.db/Tabela_Ar")
from pyspark.sql.types import *

In [10]:
spark.sql("USE Projeto")
spark.sql("SHOW tables").show()

+---------+-----------------+-----------+
|namespace|        tableName|isTemporary|
+---------+-----------------+-----------+
|  projeto|             agua|      false|
|  projeto|      tabela_agua|      false|
|  projeto|        tabela_ar|      false|
|  projeto|  tabela_petroleo|      false|
|  projeto|tabela_reciclagem|      false|
+---------+-----------------+-----------+



In [11]:
spark.table("Projeto.Tabela_Ar").toPandas()

,Unique_ID,Indicator_ID,Name,Measure,Measure_Info,Geo_Type_Name,Geo_Join_ID,Geo_Place_Name,Time_Period,Start_Date,Data_Value,End_Date,Localidade,Ano
0,179718,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,504,South Beach - Tottenville,2015,2015-01-01,2.000000,2015-12-31,Staten Island,2015
1,179719,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,503,Willowbrook,2015,2015-01-01,2.100000,2015-12-31,Staten Island,2015
2,179720,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,501,Port Richmond,2015,2015-01-01,2.800000,2015-12-31,Staten Island,2015
3,179721,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,502,Stapleton - St. George,2015,2015-01-01,4.600000,2015-12-31,Staten Island,2015
4,179722,642,Boiler Emissions- Total NOx Emissions,Number per km2,number,UHF42,410,Rockaways,2015,2015-01-01,6.100000,2015-12-31,Queens,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16117,671118,386,Ozone (O3),Mean,ppb,CD,306,Park Slope and Carroll Gardens (CD6),Summer 2020,2020-06-01,28.700001,2020-09-01,Brooklyn,2020
16118,671119,386,Ozone (O3),Mean,ppb,CD,305,East New York and Starrett City (CD5),Summer 2020,2020-06-01,29.559999,2020-09-01,Brooklyn,2020
16119,671120,386,Ozone (O3),Mean,ppb,CD,304,Bushwick (CD4),Summer 2020,2020-06-01,29.650000,2020-09-01,Brooklyn,2020
16120,671121,386,Ozone (O3),Mean,ppb,CD,303,Bedford Stuyvesant (CD3),Summer 2020,2020-06-01,29.280001,2020-09-01,Brooklyn,2020
